In [2]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
ratings = pd.read_csv('toy_dataset.csv',index_col=0)
ratings = ratings.fillna(0)
ratings

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,4.0,5.0,3.0,0.0,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,0.0
user 3,1.0,0.0,0.0,4.0,5.0,4.0
user 4,0.0,2.0,1.0,4.0,0.0,3.0
user 5,1.0,0.0,2.0,3.0,3.0,4.0


In [4]:
def standardize(row):
    new_row = (row - row.mean()) / (row.max() - row.min())
    return new_row

ratings_std = ratings.apply(standardize)
ratings_std

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,0.36,0.6,0.400000,-0.65,-0.08,-0.35
user 2,0.56,0.2,0.400000,-0.15,-0.08,-0.60
user 3,-0.24,-0.4,-0.600000,0.35,0.52,0.40
user 4,-0.44,0.0,-0.266667,0.35,-0.48,0.15
user 5,-0.24,-0.4,0.066667,0.10,0.12,0.40


In [5]:
item_similarity = cosine_similarity(ratings_std.T)
item_similarity

array([[ 1.        ,  0.70668875,  0.81368151, -0.79941088, -0.02539184,
        -0.91410609],
       [ 0.70668875,  1.        ,  0.72310153, -0.84515425, -0.5189993 ,
        -0.84337386],
       [ 0.81368151,  0.72310153,  1.        , -0.84794611, -0.3799803 ,
        -0.80218063],
       [-0.79941088, -0.84515425, -0.84794611,  1.        ,  0.14803913,
         0.72374686],
       [-0.02539184, -0.5189993 , -0.3799803 ,  0.14803913,  1.        ,
         0.39393939],
       [-0.91410609, -0.84337386, -0.80218063,  0.72374686,  0.39393939,
         1.        ]])

In [6]:
item_similarity_df = pd.DataFrame(item_similarity,index=ratings_std.columns, columns=ratings_std.columns)
item_similarity_df

,action1,action2,action3,romantic1,romantic2,romantic3
action1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
action2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
action3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
romantic1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
romantic2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
romantic3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


In [7]:
def similar_movies(movie,rating):
    similar_scores = item_similarity_df[movie] * (rating-2.5)
    similar_scores = similar_scores.sort_values(ascending=False)
    return similar_scores

similar_movies('action1',4)

action1      1.500000
action3      1.220522
action2      1.060033
romantic2   -0.038088
romantic1   -1.199116
romantic3   -1.371159
Name: action1, dtype: float64

In [13]:
dummy_user = {'action1':1,'romantic2':5,'romantic3':5}
similar_movies_df = pd.DataFrame()

for movie, rating in dummy_user.items():
    similar_movies_df = similar_movies_df.append(similar_movies(movie, rating),ignore_index=True)
    
#     similar_movies_df = pd.concat([similar_movies_df, pd.DataFrame.from_records(similar_movies(movie, rating))],)
    
# similar_movies_df.head()
similar_movies_df.sum().sort_values(ascending=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10764\1551242696.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies_df = similar_movies_df.append(similar_movies(movie, rating),ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_10764\1551242696.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies_df = similar_movies_df.append(similar_movies(movie, rating),ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_10764\1551242696.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_movies_df = similar_movies_df.append(similar_movies(movie, rating),ignore_index=True)


romantic3    4.856008
romantic2    3.522936
romantic1    3.378581
action1     -3.848745
action3     -4.175925
action2     -4.465966
dtype: float64